# <a id='toc1_'></a>[Statistics & Public Health 2: Data Analysis](#toc0_)

Sam Celarek  
Data Science   
scelarek@gmail.com  

June 4th, 2023


# <a id='toc2_'></a>[1. Introduction](#toc0_)

In this project, we will perform a set of analyses on the relationship between different variables and the mosquito number, as well as the probability of finding West Nile Virus (WNV) at any particular time and location. 


## <a id='toc2_1_'></a>[1.1. Key Questions](#toc0_)


# <a id='toc3_'></a>[2. Setup and Data Collection](#toc0_)

We will be utilizing the cleaned mosquito tracking data from the city of Chicago, Illinois, spanning from 2008 to 2019 provided [here](link_to_dataset). This section will include the necessary libraries and modules for the analysis, as well as the data preparation steps.


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
from functools import reduce

# initialize styling params
plt.rcParams['figure.figsize'] = (8.0, 6.0) #setting figure size

# Load the data

# Display the first few rows of the dataframe


In [4]:
folder_holder = '/Users/samcelarek/Documents/Github/Covid/'

# # time series data
epid_df = pd.read_csv(f'{folder_holder}epidemiology.csv') # 1
hospitalizations_df = pd.read_csv(f'{folder_holder}hospitalizations.csv') # 2
vac_df = pd.read_csv(f'{folder_holder}vaccinations.csv') # 3
mobility_df = pd.read_csv(f'{folder_holder}mobility.csv') # 4
gov_response_df = pd.read_csv(f'{folder_holder}oxford-government-response.csv') # 5
weather_df = pd.read_csv(f'{folder_holder}weather.csv') # 6

timeland_df = [epid_df, hospitalizations_df, mobility_df, vac_df, gov_response_df, weather_df]


# # location dfs
geography_df = pd.read_csv(f'{folder_holder}geography.csv') #1
health_df = pd.read_csv(f'{folder_holder}health.csv') #2
demographics_df = pd.read_csv(f'{folder_holder}demographics.csv') #3
economics_df = pd.read_csv(f'{folder_holder}economy.csv') #4
locationland_df = [geography_df, health_df, demographics_df, economics_df]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/samcelarek/Documents/Github/Covid/epidemiology.csv'

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Filter rows based on column: 'location_key'
    df = df[df['location_key'].str.contains(r"^(US|CA|DE|GB|FR|JP|AU|BR|ZA|IN)$", na=False)]
    
    # Change column type to datetime64[ns] for column: 'date'
    try:
        df = df.astype({'date': 'datetime64[ns]'})
    except:
        pass
    # Change column type to category for column: 'location_key'
    df = df.astype({'location_key': 'category'})
    return df


time_series_dfs = list(map(clean_data, timeland_df))

# Assume dfs is your list of dataframes
time_series_dfs = reduce(lambda left,right: pd.merge(left,right,on=['location_key', 'date'], how='left'), time_series_dfs).copy()


/var/folders/1h/85xzk8r97_g7dvqy0gk21fsw0000gn/T/ipykernel_16234/2871392548.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US|CA|DE|GB|FR|JP|AU|BR|ZA|IN)$", na=False)]
/var/folders/1h/85xzk8r97_g7dvqy0gk21fsw0000gn/T/ipykernel_16234/2871392548.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US|CA|DE|GB|FR|JP|AU|BR|ZA|IN)$", na=False)]
/var/folders/1h/85xzk8r97_g7dvqy0gk21fsw0000gn/T/ipykernel_16234/2871392548.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US|CA|DE|GB|FR|JP|AU|BR|ZA|IN)$", na=False)]
/var/folders/1h/85xzk8r97_g7dvqy0gk21fsw0000gn/T/ipykernel_16234/2871392548.py:6: 

In [ ]:

time_series_dfs.to_pickle('time_series_dfs.pkl')

In [ ]:
time_series_dfs.sample(3)

NameError: name 'time_series_dfs' is not defined



Here is a list of 10 countries that are often considered bellwethers for their regions and have good data collection, along with their 2-digit ISO country codes:

United States (US)  
Canada (CA)  
Germany (DE)  
United Kingdom (GB)  
France (FR)  
Japan (JP)  
Australia (AU)  
Brazil (BR)  
South Africa (ZA)  
India (IN)  
These countries are often used as indicators for their respective regions due to their significant economic influence, political stability, and comprehensive data collection practices.

In [ ]:

location_df = list(map(clean_data, locationland_df))

# Assume dfs is your list of dataframes
location_df = reduce(lambda left,right: pd.merge(left,right,on='location_key', how='left'), location_df).copy()

location_df.sample(3)


/var/folders/1h/85xzk8r97_g7dvqy0gk21fsw0000gn/T/ipykernel_16234/2871392548.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US|CA|DE|GB|FR|JP|AU|BR|ZA|IN)$", na=False)]
/var/folders/1h/85xzk8r97_g7dvqy0gk21fsw0000gn/T/ipykernel_16234/2871392548.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US|CA|DE|GB|FR|JP|AU|BR|ZA|IN)$", na=False)]
/var/folders/1h/85xzk8r97_g7dvqy0gk21fsw0000gn/T/ipykernel_16234/2871392548.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['location_key'].str.contains(r"^(US|CA|DE|GB|FR|JP|AU|BR|ZA|IN)$", na=False)]
/var/folders/1h/85xzk8r97_g7dvqy0gk21fsw0000gn/T/ipykernel_16234/2871392548.py:6: 

,location_key,openstreetmap_id,latitude,longitude,elevation_m,area_sq_km,area_rural_sq_km,area_urban_sq_km,life_expectancy,smoking_prevalence,...,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,gdp_usd,gdp_per_capita_usd,human_capital_index
5,GB,62149.0,54.6,-2.0,NaN,243610.0,183648.0,58698.0,81.30000,22.3,...,8697520.0,8872959.0,8465444.0,9020629.0,7101044.0,5567890.0,3319956.0,2.860009e+12,43070.0,0.781
2,CA,1428125.0,56.0,-109.0,NaN,9984670.0,9084045.0,126511.0,81.94878,14.3,...,5096616.0,5278661.0,4846667.0,5182433.0,4712744.0,3018676.0,1664119.0,1.736426e+12,46194.0,0.799
9,ZA,87565.0,-29.0,24.0,1037.0,1219090.0,53460.0,53460.0,63.85700,20.3,...,10141489.0,10155325.0,7043275.0,4911532.0,3164441.0,1476055.0,421794.0,3.514316e+11,6001.0,0.406


Data Wireframe:

1. Date and Location:
    - 'date'
    - 'location_key'

2. COVID-19 Statistics:
    - 'new_confirmed' (New Positive Cases)
    - 'new_deceased' (New Deaths)
    - 'new_hospitalized_patients' (New Hospitalizations)

3. Mobility Data:
    - 'mobility_retail_and_recreation'
    - 'mobility_grocery_and_pharmacy'
    - 'mobility_parks'
    - 'mobility_transit_stations'
    - 'mobility_workplaces'
    - 'mobility_residential'

4. Vaccination Data:
    - 'new_persons_vaccinated'
    - 'cumulative_persons_vaccinated'
    - 'new_persons_fully_vaccinated'
    - 'cumulative_persons_fully_vaccinated'
    - 'new_vaccine_doses_administered'
    - 'cumulative_vaccine_doses_administered'

5. Policy Measures:
    - 'school_closing'
    - 'workplace_closing'
    - 'cancel_public_events'
    - 'restrictions_on_gatherings'
    - 'public_transport_closing'
    - 'stay_at_home_requirements'
    - 'restrictions_on_internal_movement'
    - 'international_travel_controls'
    - 'income_support'
    - 'debt_relief'
    - 'fiscal_measures'
    - 'international_support'
    - 'public_information_campaigns'
    - 'testing_policy'
    - 'contact_tracing'
    - 'emergency_investment_in_healthcare'
    - 'investment_in_vaccines'
    - 'facial_coverings'
    - 'vaccination_policy'
    - 'stringency_index'

6. Weather Data:
    - 'average_temperature_celsius'
    - 'minimum_temperature_celsius'
    - 'maximum_temperature_celsius'
    - 'rainfall_mm'
    - 'snowfall_mm'
    - 'dew_point'
    - 'relative_humidity'


This dataframe provides a comprehensive snapshot of COVID-19 data, mobility metrics, government restrictions, and weather conditions for specific locations on specific dates. Here's a brief overview of the columns:

1. `Entry ID`: A unique identifier for each row in the dataframe.
2. `Date`: The date for the day on which the data was recorded.
3. `Location Key`: A code representing the location (10 different countries in total) for which the data is reported.

4. `New Confirmed`: The number of new confirmed COVID-19 cases on the given date.
5. `New Deceased`: The number of new COVID-19 related deaths on the given date.
6. `New Recovered`: The number of new recoveries from COVID-19 on the given date.
7. `New Tested`: The number of new COVID-19 tests conducted on the given date.

8. `New Hospitalizations`: The number of new hospitalizations due to COVID-19 on the given date.
9. `Current Hospitalizations`: The total number of current hospitalizations due to COVID-19 on the given date.

10. `New Fully Vaccinated (29+ other Vaccination Columns)`: The number of new fully vaccinated individuals on the given date. There are 29 other columns related to vaccination data here too.

11. `Retail and Recreation Mobility (5+ other Mobility Metrics)`: A measure of mobility in retail and recreation spaces, along with 5 other columns related to different aspects of mobility.

12. `School Closing (19+ other Government Restrictions)`: A measure indicating whether schools were closed on the given date, along with 19 other columns related to different government restrictions.

13. `Average Temp (6+ Other Weather Columns)`: The average temperature on the given date, along with 6 other columns related to different weather conditions.

In total there are 9880 and 82 rows for 6.3mbs of data. The main way I could increase or decrease the size of the dataset would be to include more countries, regions, or counties in the analysis. For now this is my starter df.
